In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Subset
import numpy as np

# 1. Kontrola dostupnosti GPU a nastavení zařízení
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 2. Načtení předtrénovaného modelu
model = models.resnet18(pretrained=True)

# 3. Odstranění poslední vrstvy a přizpůsobení modelu pro naše třídy (např. 10 tříd)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 4. Přesun modelu na GPU
model.to(device)

# 5. Nastavení datových transformací
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 6. Načtení datasetu CIFAR-10
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 7. Zamíchání dat
def shuffle_and_select(dataset, num_samples):
    # Získání indexů a zamíchání
    indices = np.arange(len(dataset))
    np.random.shuffle(indices)

    selected_indices = []
    class_counts = {i: 0 for i in range(10)}  # Počítání vzorků pro každou třídu

    for idx in indices:
        label = dataset.targets[idx]
        if class_counts[label] < num_samples // 10:  # Zajištění vyrovnaného počtu vzorků pro každou třídu
            selected_indices.append(idx)
            class_counts[label] += 1

        if len(selected_indices) >= num_samples:
            break

    return Subset(dataset, selected_indices)

# 8. Vybrání podmnožin tréninkového a testovacího datasetu
train_subset = shuffle_and_select(train_dataset, 1000)  # 1000 vzorků pro trénink
test_subset = shuffle_and_select(test_dataset, 200)  # 200 vzorků pro testování

# 9. Vytvoření DataLoaderů
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4)

# 10. Ztrátová funkce a optimalizátor
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 11. Mrazení prvních vrstev (volitelné)
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

# 12. Tréninková smyčka
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    print(f'Starting epoch {epoch + 1}')
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)  # Přesun dat na GPU
        optimizer.zero_grad()  # Vyčištění gradientů
        outputs = model(images)  # Predikce
        loss = criterion(outputs, labels)  # Výpočet ztráty
        loss.backward()  # Výpočet gradientů
        optimizer.step()  # Aktualizace parametrů

        if i % 100 == 0:  # Každých 100 batchů
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i}], Loss: {loss.item():.4f}')

# 13. Vyhodnocení modelu
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Přesun dat na GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')



Using device: cpu
Files already downloaded and verified
Files already downloaded and verified
Starting epoch 1


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1/5], Batch [0], Loss: 2.2960
Starting epoch 2
Epoch [2/5], Batch [0], Loss: 1.6589
Starting epoch 3
Epoch [3/5], Batch [0], Loss: 1.3132
Starting epoch 4
Epoch [4/5], Batch [0], Loss: 0.8647
Starting epoch 5
Epoch [5/5], Batch [0], Loss: 0.7732
Accuracy of the model on the test images: 67.00%
